In [1]:
from keras.models import Model 
from keras.layers import Input, Convolution2D, MaxPooling2D, Dense, Dropout, Flatten, Dense, Dropout, Flatten, Conv2D, MaxPooling2D
import np_utils
import numpy as np
import pandas as pd
from keras.callbacks import EarlyStopping
from keras.models import Sequential
from keras.optimizers import Adam
from keras.utils import to_categorical
from sklearn.preprocessing import StandardScaler
import os
import datetime
from sklearn.model_selection import train_test_split

In [2]:
poor = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_poor.csv", header=None)
middle = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_middle.csv", header=None)
rich = pd.read_csv("C:/Users/met48/Desktop/TS-Clustering/SimData/bank_reserves_outputs_rich.csv", header=None)
sc = StandardScaler()
bank_reserves = []
for i in np.arange(0, poor.shape[0]):
    sample = pd.concat([poor.iloc[i], middle.iloc[i]], axis=0).T
    sample = pd.concat([sample, rich.iloc[i]], axis=0).T
    sample_std = sc.fit_transform(sample.to_frame())
    bank_reserves.append(sample_std)

In [3]:
bank_reserves = [sample.flatten() for sample in bank_reserves]

In [4]:
bank_reserves = np.asarray(bank_reserves)

In [5]:
bank_reserves

array([[-1.36670362, -1.36670362, -1.36670362, ..., -0.18244118,
        -0.15706413, -0.12322806],
       [-1.31428663, -1.31428663, -1.31428663, ..., -0.14173205,
        -0.11730383, -0.10101835],
       [-1.07779326, -1.07779326, -1.07779326, ..., -0.27756778,
        -0.29079465, -0.22466031],
       ...,
       [-2.12830655, -2.12830655, -2.12830655, ...,  0.14970836,
         0.13646409,  0.10997555],
       [-1.76787358, -1.76787358, -1.76787358, ..., -0.11811475,
        -0.05212439,  0.06885792],
       [-1.62916518, -1.62916518, -1.62916518, ..., -0.09220556,
        -0.08209399, -0.15287502]])

In [6]:
bank_reserves.shape

(100000, 303)

In [7]:
bank_reserves_train, bank_reserves_test = train_test_split(bank_reserves, test_size=0.20, random_state=42)

In [8]:
input = Input(shape=(303,))

encoded = Dense(30, activation='relu')(input)
encoded = Dense(20, activation='relu')(encoded)
encoded = Dense(10, activation='relu')(encoded)
encoded = Dense(5, activation='relu')(encoded)

decoded = Dense(10, activation='relu')(encoded)
decoded = Dense(20, activation='relu')(decoded)
decoded = Dense(30, activation='relu')(decoded)
output = Dense(303, activation=None)(decoded)

autoencoder = Model(input, output)
autoencoder.summary()

Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)        │ (None, 303)            │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 30)             │         9,120 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 20)             │           620 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_2 (Dense)                 │ (None, 10)             │           210 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_3 (Dense)                 │ (None, 5)              │            55 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_4 (Dense)                 │ (None, 10)             │            60 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_5 (Dense)                 │ (None, 20)             │           220 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_6 (Dense)                 │ (None, 30)             │           630 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_7 (Dense)                 │ (None, 303)            │         9,393 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 20,308 (79.33 KB)

 Trainable params: 20,308 (79.33 KB)

 Non-trainable params: 0 (0.00 B)

In [9]:
autoencoder.compile(optimizer=Adam(learning_rate=0.001), loss='mse')

In [10]:
autoencoder.fit(bank_reserves_train, bank_reserves_train,
 epochs=20,
 batch_size=256,
 shuffle=True,
 validation_data=(bank_reserves_test, bank_reserves_test))

Epoch 1/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 3s 2ms/step - loss: 0.3684 - val_loss: 0.0100
Epoch 2/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0095 - val_loss: 0.0079
Epoch 3/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0076 - val_loss: 0.0072
Epoch 4/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0071 - val_loss: 0.0065
Epoch 5/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0063 - val_loss: 0.0060
Epoch 6/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0060 - val_loss: 0.0059
Epoch 7/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0059 - val_loss: 0.0058
Epoch 8/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0058 - val_loss: 0.0057
Epoch 9/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0057 - val_loss: 0.0056
Epoch 10/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0056 - val_loss: 0.0054
Epoch 11/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step - loss: 0.0053 - val_loss: 0.0051
Epoch 12/50
313/313 ━━━━━━━━━━━━━━━━━━━━ 1s 2ms/step

In [11]:
encoder = Model(inputs=autoencoder.input, outputs=autoencoder.get_layer(index=4).output)
encoded_ts = encoder.predict(bank_reserves_test)

625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 612us/step


In [12]:
encoded_input = Input(shape=(5,))  # 5 is the size of your latent space
decoded_layer = autoencoder.layers[5](encoded_input)
for layer in autoencoder.layers[6:]:
    decoded_layer = layer(decoded_layer)

decoder = Model(inputs=encoded_input, outputs=decoded_layer)
decoded_ts = decoder.predict(encoded_ts)

625/625 ━━━━━━━━━━━━━━━━━━━━ 0s 632us/step


In [13]:
decoded_ts[0]

array([-1.152258  , -1.149972  , -1.151139  , -1.1477269 , -1.1391687 ,
       -1.1318135 , -1.1260916 , -1.1157244 , -1.1095328 , -1.1070526 ,
       -1.1051044 , -1.1049978 , -1.1008574 , -1.0994409 , -1.097996  ,
       -1.0947391 , -1.09131   , -1.0900297 , -1.0970626 , -1.0949835 ,
       -1.0946628 , -1.0887686 , -1.0924032 , -1.092135  , -1.0897561 ,
       -1.0929116 , -1.092159  , -1.0905619 , -1.0893824 , -1.0860204 ,
       -1.0868268 , -1.0844616 , -1.0768942 , -1.084673  , -1.0851805 ,
       -1.0791315 , -1.0863115 , -1.0820857 , -1.0759581 , -1.0706844 ,
       -1.0723836 , -1.0730183 , -1.0725251 , -1.0737731 , -1.081433  ,
       -1.0805476 , -1.0798911 , -1.081267  , -1.0824631 , -1.0781085 ,
       -1.0759039 , -1.074195  , -1.0781646 , -1.0786783 , -1.074559  ,
       -1.0779976 , -1.0739659 , -1.0788033 , -1.0826796 , -1.0781418 ,
       -1.073845  , -1.0848088 , -1.0798458 , -1.0783339 , -1.0719131 ,
       -1.0769876 , -1.0763143 , -1.0757017 , -1.0824742 , -1.07

In [14]:
bank_reserves_test[0]

array([-1.15390451, -1.15390451, -1.15390451, -1.14678831, -1.13255592,
       -1.13255592, -1.13967212, -1.14678831, -1.13967212, -1.13255592,
       -1.12543972, -1.11832352, -1.12543972, -1.13255592, -1.13967212,
       -1.13255592, -1.12543972, -1.12543972, -1.12543972, -1.12543972,
       -1.11832352, -1.11120732, -1.08985873, -1.07562633, -1.09697492,
       -1.08274253, -1.08985873, -1.10409112, -1.10409112, -1.10409112,
       -1.10409112, -1.08985873, -1.08274253, -1.06851013, -1.06139393,
       -1.06851013, -1.08985873, -1.08274253, -1.07562633, -1.07562633,
       -1.06851013, -1.08985873, -1.08274253, -1.07562633, -1.06851013,
       -1.08274253, -1.07562633, -1.07562633, -1.08274253, -1.08274253,
       -1.08274253, -1.07562633, -1.07562633, -1.06851013, -1.08274253,
       -1.08274253, -1.08985873, -1.08985873, -1.09697492, -1.10409112,
       -1.11120732, -1.08274253, -1.07562633, -1.08985873, -1.08274253,
       -1.08274253, -1.08274253, -1.06851013, -1.06851013, -1.06

In [19]:
encoded_ts = encoder.predict(bank_reserves)

3125/3125 ━━━━━━━━━━━━━━━━━━━━ 2s 582us/step


In [16]:
type(encoded_ts)

numpy.ndarray

In [20]:
np.savetxt("C:/Users/met48/Desktop/TS-Clustering/extracted_features/bank_reserves_DAE.csv", encoded_ts, delimiter=",")